This notebook builds a smart finance assistant chatbot that leverages AI to help users track and manage their personal transactions. The chatbot extracts financial details from user input using a Language Model (LLM), categorizes income and expenses, and logs them into a database. It validates entries, and provides insightful feedback about spending habits. By combining natural language understanding with real-time database integration, this chatbot offers a seamless and intelligent way to automate personal finance management and budgeting.

## 1- Import Libraries 

In [42]:
import os
from langchain_fireworks import Fireworks
from langchain.memory import ConversationBufferMemory
from datetime import datetime
from langchain.retrievers import EnsembleRetriever
import re
from dateparser import parse as date_parse
from datetime import datetime
from supabase import create_client, Client
from dotenv import load_dotenv

## 2- Set API key 

In [ ]:
# Load the .env file
load_dotenv()

# Access the variables
api_key = os.getenv("API_KEY")

## 3- Call LLM 

In [44]:
llm = Fireworks(api_key=api_key, model="accounts/fireworks/models/deepseek-v3", temperature=1.0, max_tokens=1024)
response = llm.invoke("Hello, how are you?")
print(response)

 My name is Antonio, I’m a musician from Lisbon and I have a passion for teaching. My academic education started in a classical music school in which I studied for 10 years, where the main focus was in flute, piano and other music related subjects such as theory, history or composition. After that I picked up the viola (viola is a different instrument from Viola. Viola as an instrument is a bit bigger than a violin, with a deeper and more muffled sound). After studying classical viola for a few years I found that my preferences in music style were leading me to other places. I spent a few years in the Conservatorium of Amsterdam learning and playing different styles of music, using the viola, violin, guitar, piano and singing, as well as experimenting with electronic music production and world music. I’ve been teaching classical piano, viola and violin for 7 years, where I have experienced teaching people of all ages (3 to 60 years old) and levels (beginner to Master level). I’m also f

## 4- Create memory

In [45]:
#  Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


## 5- Fetch database

In [ ]:
# Load .env
load_dotenv()
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")

# Connect to Supabase
supabase: Client = create_client(url, key)

In [47]:
# Fetch data from the "Transaction" table
response = supabase.table("transactions").select("*").execute()
data = response.data
print(data)

[{'transaction_id': '3683c4fc-018d-42ee-803b-69845dd1f0cd', 'user_id': '027051a8-3887-4150-9cfb-a51efb9146b5', 'income': 7000, 'expenses': 0, 'data': 'Monthly salary deposit', 'category_id': '8867173b-22a3-408e-a2b8-9ee2f0bc70b2', 'description': 'August salary', 'created_at': '2025-04-21T10:51:09.833846'}, {'transaction_id': '2f662481-3b4e-47e3-aeec-f0ca06c7c7fb', 'user_id': 'eb242352-2899-4717-955c-7247db8a40ef', 'income': 0, 'expenses': 150.75, 'data': None, 'category_id': '813c1667-760d-46c2-8226-dc139cc8c86c', 'description': 'Bought groceries from Carrefour', 'created_at': '2025-05-07T09:50:25.344434'}, {'transaction_id': 'bd7bd415-064b-4d9c-a0c3-4942e5fd1a43', 'user_id': 'eb242352-2899-4717-955c-7247db8a40ef', 'income': 0, 'expenses': 250, 'data': None, 'category_id': '114ed927-ca6b-47df-bf3d-689ff3b8bbd9', 'description': 'Weekly groceries at Carrefour', 'created_at': '2025-05-07T09:53:27.097738'}, {'transaction_id': 'f7441b54-0fe6-4772-ad92-dcf1c64c44cd', 'user_id': 'eb242352-289

In [48]:
# Fetch data from the "categories" table
response = supabase.table("categories").select("*").execute()
data = response.data
print(data)

[{'category_id': '337bf3ed-8f2e-4647-ac50-30f327316d07', 'category_type': 'Food', 'category_name': 'Caffe', 'user_id': None}, {'category_id': 'c45b8dff-e08f-4c36-b30c-68ae7026b2f8', 'category_type': 'Transportation', 'category_name': 'taxi', 'user_id': None}, {'category_id': '3782b789-59fa-45b1-a214-c29e21a2f0e3', 'category_type': 'transportation', 'category_name': 'fuel', 'user_id': None}, {'category_id': '8867173b-22a3-408e-a2b8-9ee2f0bc70b2', 'category_type': 'Expense', 'category_name': 'Groceries', 'user_id': '027051a8-3887-4150-9cfb-a51efb9146b5'}, {'category_id': 'a8a017fc-e86f-4c23-9455-4f8db846c59b', 'category_type': 'Income', 'category_name': 'Salary', 'user_id': '47b66d92-1ea1-4344-af56-0771ed917456'}, {'category_id': 'af91ae83-0d65-4bd1-880f-747f3ebbaf35', 'category_type': 'Expense', 'category_name': 'Utilities', 'user_id': '99eed9c3-78dc-4925-b599-b02332524f78'}, {'category_id': '813c1667-760d-46c2-8226-dc139cc8c86c', 'category_type': 'expense', 'category_name': 'Groceries'

## 6- Prompt

In [49]:
def process_transaction_with_llm(text):
    # Prompt for extracting transaction details
    transaction_prompt = """
    Extract transaction details from the following text and classify the amount as either income or expense.
    If it's an expense, put amount in EXPENSE; if it’s income, put amount in INCOME. Extract date (handle relative dates like 'yesterday'), a description, and the category.
    If a specific date is not mentioned, assume today's date. If you cant classify the amount as either income or expense,
    assume it is expense and keep it uncategorized. Provide a natural response about the spending.

    Text: {input_text}

    Current date: {current_date}

    Format your response as follows:
    DATE: [Extract the date, handle relative dates like 'this morning', 'yesterday', 'last week', etc.]
    EXPENSE: [amount, if spending; leave blank if not]
    INCOME: [amount, if income; leave blank if not]
    CATEGORY: [Classify into one of if expense: Food, Health, Transportation, Entertainment, Fashion, Lifestyle, Education
               Classify into one of if income: Salary, Part-time, Investments, Bonus, Refunds]
    Description: [one or two word description of the transaction]
    FEEDBACK: [Provide a natural, conversational response about the spending. Consider:
    - If it's a good deal or expensive for that category
    - Suggest money-saving tips if relevant
    - Compliment good financial decisions
    - Express concern for unusually high spending
    - Comment on the timing or necessity of the purchase
    Make it sound natural and varied.]

    Example:
    Input: "I got today 2,000 EGP from upwork"
    DATE: 2024-01-30
    INCOME: 2000
    CATEGORY: Part-time
    DESCRIPTION: upwork
    FEEDBACK: That’s wonderful—congrats on your part-time job!
    
    Input: "Bought lunch for 50 EGP yesterday"
    DATE: 2024-01-29
    EXPENSE: 50
    CATEGORY: Food
    DESCRIPTION: lunch
    FEEDBACK: That's a reasonable amount for lunch! If you're looking to save more, you might consider bringing lunch from home occasionally.
    """

    # Prepare the prompt with current date
    current_date = datetime.now().strftime('%Y-%m-%d')
    formatted_prompt = transaction_prompt.format(
        input_text=text,
        current_date=current_date
    )

    # Get response from LLM
    response = llm.invoke(formatted_prompt)
    
    # Parse the response (you might need to adjust this based on your LLM's output format)
    return response



In [50]:
print(process_transaction_with_llm("Had dinner for 150 EGP last night"))


 

    DATE: 2025-05-09
    EXPENSE: 150
    CATEGORY: Food
    DESCRIPTION: dinner
    FEEDBACK: 150 EGP for dinner is a decent amount, but it seems like you had a good time! If you're dining out frequently, you could explore more budget-friendly options to save some money.


In [51]:
print(process_transaction_with_llm("got my salary 6000"))


## 7- Parse response

In [55]:
import re
from datetime import datetime
from dateparser import parse as date_parse

def parse_llm_response(response):
    # Extract date (default to today if not found or "Unknown")
    date_match = re.search(r"DATE:\s*(.*)", response)
    date_str = date_match.group(1).strip() if date_match else None
    if not date_str or date_str.lower() == "unknown":
        date_obj = datetime.now()
    else:
        date_obj = date_parse(date_str)
        if not date_obj:
            date_obj = datetime.now()
    date_iso = date_obj.strftime("%Y-%m-%d")

    # Extract EXPENSE and INCOME
    expense_match = re.search(r"EXPENSE:\s*([\d.]+)", response, re.IGNORECASE)
    income_match = re.search(r"INCOME:\s*([\d.]+)", response, re.IGNORECASE)
    expense = float(expense_match.group(1)) if expense_match else 0.0
    income = float(income_match.group(1)) if income_match else 0.0

    # Extract DESCRIPTION
    desc_match = re.search(r"DESCRIPTION:\s*(.*)", response, re.IGNORECASE)
    description = desc_match.group(1).strip() if desc_match and desc_match.group(1).strip() else None

    # Extract CATEGORY (optional)
    category_match = re.search(r"CATEGORY:\s*(.*)", response)
    category = category_match.group(1).strip() if category_match and category_match.group(1).strip() else None

    # Extract FEEDBACK (optional)
    feedback_match = re.search(r"FEEDBACK:\s*(.*)", response)
    feedback = feedback_match.group(1).strip() if feedback_match else ""

    return {
        "date": date_iso,
        "expense": expense,
        "income": income,
        "category": category,
        "description": description,
        "feedback": feedback
    }



## 8- Get category_id

In [56]:
def get_category_id(category_name: str, supabase):
    # Returns None if category_name is None or not found
    if not category_name:
        return None
    cat_response = supabase.table("categories").select("category_id").eq("category_name", category_name).execute()
    cat_data = cat_response.data
    if cat_data:
        return cat_data[0]["category_id"]
    else:
        return None


## 9- Add transaction into DB

In [57]:
def add_transaction_from_llm(llm_response, user_id, supabase):
    extracted = parse_llm_response(llm_response)
    description_valid = extracted["description"] not in (None, "", "No Description")
    amount_valid = (extracted["expense"] > 0 or extracted["income"] > 0)

    if not description_valid or not amount_valid:
        return "❌ Cannot add transaction: missing information."

    category_id = get_category_id(extracted["category"], supabase)
    new_transaction = {
        "income": extracted["income"],
        "expenses": extracted["expense"],
        "description": extracted["description"],
        "created_at": extracted["date"],
        "user_id": user_id,
        "category_id": category_id
    }
    response = supabase.table("transactions").insert(new_transaction).execute()

    # Build friendly message for user
    # Compose a string stating what was added, and echo the LLM's feedback
    tx_type = "income" if extracted["income"] > 0 else "expense"
    tx_value = extracted["income"] if extracted["income"] > 0 else extracted["expense"]

    user_reply = (
        f"✅ Added {tx_type} transaction: {extracted['description']}, "
        f"{tx_value} EGP, "
        f"date: {extracted['date']}"
    )
    if extracted["category"]:
        user_reply += f", category: {extracted['category']}."
    else:
        user_reply += "."
    if extracted["feedback"]:
        user_reply += f"\n💬 {extracted['feedback']}"
    return user_reply

## 10- Test

In [58]:
user_input = "Received salary 8000 EGP today"
llm_response = process_transaction_with_llm(user_input) 
print(llm_response)
result = add_transaction_from_llm(llm_response, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase)
print(result)

 

    DATE: 2025-05-10  
    INCOME: 8000  
    CATEGORY: Salary  
    DESCRIPTION: salary  
    FEEDBACK: That’s great—your salary has been credited! Make sure to plan your finances wisely to meet your goals.


C:\Users\Tony\anaconda3\lib\site-packages\tzlocal\utils.py:39: UserWarning: Timezone offset does not match system offset: 7200 != 10800. Please, check your config files.
  warnings.warn(msg)


✅ Added income transaction: salary, 8000.0 EGP, date: 2025-05-10, category: Salary.
💬 That’s great—your salary has been credited! Make sure to plan your finances wisely to meet your goals.


In [59]:
user_input = "Had dinner for 150 EGP last night"
llm_response = process_transaction_with_llm(user_input)
print(llm_response)
# Parse and save
add_transaction_from_llm(llm_response, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase)

 DATE: 2025-05-09
     EXPENSE: 150
     CATEGORY: Food
     DESCRIPTION: dinner
     FEEDBACK: Dining out can be a treat, but 150 EGP for dinner seems a bit high. Consider cooking at home more often to save money while still enjoying a good meal.


C:\Users\Tony\anaconda3\lib\site-packages\tzlocal\utils.py:39: UserWarning: Timezone offset does not match system offset: 7200 != 10800. Please, check your config files.
  warnings.warn(msg)


'✅ Added expense transaction: dinner, 150.0 EGP, date: 2025-05-09, category: Food.\n💬 Dining out can be a treat, but 150 EGP for dinner seems a bit high. Consider cooking at home more often to save money while still enjoying a good meal.'

In [61]:
user_input = "got 150 EGP"
llm_response = process_transaction_with_llm(user_input)
print(llm_response)
# Parse and save
result = add_transaction_from_llm(llm_response, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase)
print(result)





    DATE: 2025-05-10
    INCOME: 150
    CATEGORY: Uncategorized
    DESCRIPTION: got
    FEEDBACK: You received 150 EGP—great! Use it wisely or consider saving it for future needs.


C:\Users\Tony\anaconda3\lib\site-packages\tzlocal\utils.py:39: UserWarning: Timezone offset does not match system offset: 7200 != 10800. Please, check your config files.
  warnings.warn(msg)


✅ Added income transaction: got, 150.0 EGP, date: 2025-05-10, category: Uncategorized.
💬 You received 150 EGP—great! Use it wisely or consider saving it for future needs.


In [62]:
user_input = "Dinner today"
llm_response = process_transaction_with_llm(user_input)
print(llm_response)
# Parse and save
result = add_transaction_from_llm(llm_response, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase)
print(result)




    DATE: 2025-05-10  
    EXPENSE:  
    CATEGORY: Food  
    DESCRIPTION: Dinner  
    FEEDBACK: Dinner is always a great way to end the day. If you're trying to save, you might consider cooking at home sometimes—it’s often healthier and more cost-effective. Enjoy your meal!
❌ Cannot add transaction: missing amount and/or description.


C:\Users\Tony\anaconda3\lib\site-packages\tzlocal\utils.py:39: UserWarning: Timezone offset does not match system offset: 7200 != 10800. Please, check your config files.
  warnings.warn(msg)


In [63]:
user_input = "50 today"
llm_response = process_transaction_with_llm(user_input)
print(llm_response)
# Parse and save
result = add_transaction_from_llm(llm_response, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase)
print(result)

 

    DATE: 2025-05-10
    EXPENSE: 50
    CATEGORY: Uncategorized
    DESCRIPTION: today
    FEEDBACK: Hmm, I’m not entirely sure what this 50 was for. It’s always a good idea to keep track of your spending to better understand where your money goes.


C:\Users\Tony\anaconda3\lib\site-packages\tzlocal\utils.py:39: UserWarning: Timezone offset does not match system offset: 7200 != 10800. Please, check your config files.
  warnings.warn(msg)


✅ Added expense transaction: today, 50.0 EGP, date: 2025-05-10, category: Uncategorized.
💬 Hmm, I’m not entirely sure what this 50 was for. It’s always a good idea to keep track of your spending to better understand where your money goes.
